In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
## CPU
if torch.cuda.is_available()==False:
    device = torch.device("cpu")
    deviceName = "cpu"
## GPU
else:
    device = torch.device("cuda:0")
    deviceName = "cuda:0"
print('cuda is available :',torch.cuda.is_available())
print('deviceName :',deviceName)

cuda is available : True
deviceName : cuda:0


In [5]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/kongminseok/hdd/2022_kbSecurities_DIM/.cache/kobert_v1.zip
using cached model. /home/kongminseok/hdd/2022_kbSecurities_DIM/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [6]:
# 학습용 데이터셋 불러오기
import pandas as pd
use_data = pd.read_csv('./data/fineTuningData.csv')
#use_data = use_data[:1000]
use_data.tail()

,title,label
51852,"하이투자증권, ELS 2종 공모 최대 13.20 수익 지급",1
51853,"미래에셋운용, 글로벌ESG 주식 투자 상생ESG 펀드 출시",1
51854,"특징주 대성엘텍, 현대모비스 M A 후보 로 부각 5 상승",2
51855,주식 초고수는 지금 HMM에 쏠린 매수 현대바이오 부광 등 거래 상위권,1
51856,"GC녹십자셀, 국내 최초 세포치료제 전문 CDMO 도약",1


In [7]:
# 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(use_data['label'])
use_data['label'] = encoder.transform(use_data['label'])
use_data.tail()

,title,label
51852,"하이투자증권, ELS 2종 공모 최대 13.20 수익 지급",1
51853,"미래에셋운용, 글로벌ESG 주식 투자 상생ESG 펀드 출시",1
51854,"특징주 대성엘텍, 현대모비스 M A 후보 로 부각 5 상승",2
51855,주식 초고수는 지금 HMM에 쏠린 매수 현대바이오 부광 등 거래 상위권,1
51856,"GC녹십자셀, 국내 최초 세포치료제 전문 CDMO 도약",1


In [8]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 0, 1: 1, 2: 2}

In [9]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(use_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 41485
test shape is: 10372


In [10]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/kongminseok/hdd/2022_kbSecurities_DIM/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
        #self.sentences = [transform([i]) for i in dataset[dataset.columns[sent_idx]]]
        #self.labels = [np.int32(i) for i in dataset[dataset.columns[label_idx]]]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
# 파라미터 설정
max_len = 200 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음, default=64
batch_size = 16 # default = 64
warmup_ratio = 0.1
num_epochs = 5 # default = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5 # default=5e-5

In [13]:
train = train.values
test = test.values

In [14]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [15]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3, # 3가지로 분류
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [18]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [19]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/home/kongminseok/anaconda3/envs/kobert/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [21]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [22]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [23]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [24]:
from os import TMP_MAX
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
        gc.collect()
        torch.cuda.empty_cache()
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
        gc.collect()
        torch.cuda.empty_cache()
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    torch.save(model,f'./model/model_{deviceName}.pt')
    torch.save(model,f'./model/model_{deviceName}.pth')

  0%|          | 1/2593 [00:00<14:59,  2.88it/s]

epoch 1 batch id 1 loss 1.268817663192749 train acc 0.125


  8%|▊         | 201/2593 [00:59<11:56,  3.34it/s]

epoch 1 batch id 201 loss 0.8461461067199707 train acc 0.6106965174129353


 15%|█▌        | 401/2593 [01:59<11:02,  3.31it/s]

...

epoch 5 batch id 2401 loss 0.15865473449230194 train acc 0.9460641399416909


100%|██████████| 2593/2593 [14:38<00:00,  2.95it/s]

epoch 5 train acc 0.9465869649055149



100%|██████████| 649/649 [01:39<00:00,  6.52it/s]


epoch 5 test acc 0.8580508474576272
